In [1]:
import Chess_pieces
import pygame
import chess
import math
import random
from copy import deepcopy
import chess.polyglot
#from AI import random_agent

pygame 2.4.0 (SDL 2.26.4, Python 3.7.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
reader = chess.polyglot.open_reader('baron30.bin')

In [3]:
# Resize the image to a specific size
new_size = (90, 90)  # Set the desired size
#names
b_pawn = pygame.transform.scale(pygame.image.load('Chess_pieces//b_pawn.png'), new_size)
b_knight = pygame.transform.scale(pygame.image.load('Chess_pieces//b_knight.png'), new_size)
b_bishop = pygame.transform.scale(pygame.image.load('Chess_pieces//b_bishop.png'), new_size)
b_rook = pygame.transform.scale(pygame.image.load('Chess_pieces//b_rook.png'), new_size)
b_queen = pygame.transform.scale(pygame.image.load('Chess_pieces//b_queen.png'), new_size)
b_king = pygame.transform.scale(pygame.image.load('Chess_pieces//b_king.png'), new_size)
w_pawn = pygame.transform.scale(pygame.image.load('Chess_pieces//w_pawn.png'), new_size)
w_knight = pygame.transform.scale(pygame.image.load('Chess_pieces//w_knight.png'), new_size)
w_bishop = pygame.transform.scale(pygame.image.load('Chess_pieces//w_bishop.png'), new_size)
w_rook = pygame.transform.scale(pygame.image.load('Chess_pieces//w_rook.png'), new_size)
w_queen = pygame.transform.scale(pygame.image.load('Chess_pieces//w_queen.png'), new_size)
w_king = pygame.transform.scale(pygame.image.load('Chess_pieces/w_king.png'), new_size)

In [4]:
#initialise display
X = 800
Y = 800
scrn = pygame.display.set_mode((X, Y))

#basic colours
WHITE = (255, 255, 255)
GREY = (128, 128, 128)
YELLOW = (204, 204, 0)
BLUE = (50, 255, 255)
BLACK = (0, 0, 0)

#initialise chess board
pygame.init()
b = chess.Board()

#load piece images
pieces = {'p': b_pawn.convert(),
          'n': b_knight.convert(),
          'b': b_bishop.convert(),
          'r': b_rook.convert(),
          'q': b_queen.convert(),
          'k': b_king.convert(),
          'P': w_pawn.convert(),
          'N': w_knight.convert(),
          'B': w_bishop.convert(),
          'R': w_rook.convert(),
          'Q': w_queen.convert(),
          #'K': pygame.image.load('Chess_pieces//w_king.png').convert(),
          'K': w_king.convert()
          
          }

In [5]:
def update(scrn, board):
    scrn.fill(BLACK)

    for i in range(64):
        piece = board.piece_at(i)
        if piece is None:
            continue
        else:
            piece_image = pieces[piece.symbol()]
            piece_rect = piece_image.get_rect()

            # Calculate the position to center the piece
            x = (i % 8) * 100 + (100 - piece_rect.width) // 2
            y = 700 - (i // 8) * 100 + (100 - piece_rect.height) // 2

            scrn.blit(piece_image, (x, y))

    for i in range(7):
        i = i + 1
        pygame.draw.line(scrn, WHITE, (0, i * 100), (800, i * 100))
        pygame.draw.line(scrn, WHITE, (i * 100, 0), (i * 100, 800))

    pygame.display.flip()

In [39]:
def main(BOARD):

    '''
    for human vs human game
    '''
    #make background black
    scrn.fill(BLACK)
    #name window
    pygame.display.set_caption('Chess')
    
    #variable to be used later
    index_moves = []

    status = True
    while (status):
        #update screen
        update(scrn,BOARD)

        for event in pygame.event.get():
     
            # if event object type is QUIT
            # then quitting the pygame
            # and program both.
            if event.type == pygame.QUIT:
                status = False

            # if mouse clicked
            if event.type == pygame.MOUSEBUTTONDOWN:
                #remove previous highlights
                scrn.fill(BLACK)
                #get position of mouse
                pos = pygame.mouse.get_pos()

                #find which square was clicked and index of it
                square = (math.floor(pos[0]/100),math.floor(pos[1]/100))
                index = (7-square[1])*8+(square[0])
                
                # if we are moving a piece
                if index in index_moves: 
                    
                    move = moves[index_moves.index(index)]
                    
                    BOARD.push(move)

                    #reset index and moves
                    index=None
                    index_moves = []
                    
                    
                # show possible moves
                else:
                    #check the square that is clicked
                    piece = BOARD.piece_at(index)
                    #if empty pass
                    if piece == None:
                        
                        pass
                    else:
                        
                        #figure out what moves this piece can make
                        all_moves = list(BOARD.legal_moves)
                        moves = []
                        for m in all_moves:
                            if m.from_square == index:
                                
                                moves.append(m)

                                t = m.to_square

                                TX1 = 100*(t%8)
                                TY1 = 100*(7-t//8)

                                
                                #highlight squares it can move to
                                pygame.draw.rect(scrn,BLUE,pygame.Rect(TX1,TY1,100,100),5)
                        
                        index_moves = [a.to_square for a in moves]
     
    # deactivates the pygame library
        if BOARD.outcome() != None:
            print(BOARD.outcome())
            status = False
            print(BOARD)
    pygame.quit()

In [9]:
scoring= {'p': -1,
          'n': -3,
          'b': -3,
          'r': -5,
          'q': -9,
          'k': 0,
          'P': 1,
          'N': 3,
          'B': 3,
          'R': 5,
          'Q': 9,
          'K': 0,
          }

In [6]:
#def eval_board(BOARD):
#    score = 0
#    pieces = BOARD.piece_map()
#    for key in pieces:
#        score += scoring[str(pieces[key])]
#
#    return score
def eval_space(BOARD):
    no_moves = len(list(BOARD.legal_moves))

    #this function is always between 0 and 1 so we will never evaluate
    #this as being greater than a pawns value. The 20 value is arbitrary
    #but this number is chosen as it centers value around 0.5
    value = (no_moves/(20+no_moves))
    
    if BOARD.turn == True:
        return value
    else:
        return -value

In [7]:
#this is min_max at depth one
def most_value_agent(BOARD):

    moves = list(BOARD.legal_moves)
    scores = []
    for move in moves:
        #creates a copy of BOARD so we dont
        #change the original class
        temp = deepcopy(BOARD)
        temp.push(move)

        scores.append(eval_board(temp))

    if BOARD.turn == True:
        best_move = moves[scores.index(max(scores))]

    else:
        best_move = moves[scores.index(min(scores))]

    return best_move

In [8]:
def min_maxN(BOARD,N):
    moves = list(BOARD.legal_moves)
    scores = []

    for move in moves:
        temp = deepcopy(BOARD)
        temp.push(move)

        if N>1:
            temp_best_move = min_maxN(temp,N-1)
            temp.push(temp_best_move)

        scores.append(eval_board(temp))

    if BOARD.turn == True:
       
        best_move = moves[scores.index(max(scores))]

    else:
        best_move = moves[scores.index(min(scores))]

    return best_move
        
# a simple wrapper function as the display only gives one imput , BOARD
def play_min_maxN(BOARD):
    N=3
    return min_maxN(BOARD,N)

In [105]:
def random_agent(board):
    legal_moves = list(board.legal_moves)
    move = random.choice(legal_moves)
    return move


In [6]:
def play_game(board, agent, agent_color):
    index_moves = []
    highlighted_squares = []  # List to store highlighted squares

    # Main game loop
    while True:
        # Update the screen
        #possible move fix
        if not highlighted_squares:
            update(scrn, board)

        # Check for game over conditions
        if board.is_game_over():
            print("Game Over")
            print("Result: " + board.result())
            break

        # Agent's turn
        if board.turn == agent_color:
            move = agent(board)  # Generate a move using the agent function
            board.push(move)  # Push the generated move to the board
            # pygame.time.wait(500)  # Delay for visualization (optional)

        # Human player's turn
        else:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    return

                if event.type == pygame.MOUSEBUTTONDOWN:
                    #update screen if i click mouse down so it doesnt show the possible move for every piece
                    update(scrn, board)
                    pos = pygame.mouse.get_pos()
                    square = (math.floor(pos[0] / 100), math.floor(pos[1] / 100))
                    index = (7 - square[1]) * 8 + square[0]
                    highlighted_squares = []  # Clear the highlighted squares
                    
                    if index in index_moves:
                        move = moves[index_moves.index(index)]
                        board.push(move)
                        index_moves = []
                        highlighted_squares = []  # Clear the highlighted squares
                    else:
                        piece = board.piece_at(index)
                        if piece is not None:
                            all_moves = list(board.legal_moves)
                            moves = []
                            for m in all_moves:
                                if m.from_square == index:
                                    moves.append(m)
                                    t = m.to_square
                                    highlighted_squares.append((t % 8, 7 - t // 8))  # Add highlighted square coordinates
                            index_moves = [m.to_square for m in moves]

        # Draw highlighted squares
        for square in highlighted_squares:
            TX1 = 100 * square[0]
            TY1 = 100 * square[1]
            pygame.draw.rect(scrn, BLUE, pygame.Rect(TX1, TY1, 100, 100), 5)

            pygame.display.flip()

In [15]:
scrn = pygame.display.set_mode((X, Y))

In [14]:
play_game(b, min_max2, chess.BLACK, reader)

min_maxN called with N = 2
Found opening move: f7f5
min_maxN called with N = 2
Found opening move: f5g4
min_maxN called with N = 2
Recursing with N = 1
min_maxN called with N = 1
Returning best move: c1h6
Recursing with N = 1
min_maxN called with N = 1
Returning best move: c1h6
Recursing with N = 1
min_maxN called with N = 1
Returning best move: c1g5
Recursing with N = 1
min_maxN called with N = 1
Returning best move: c1h6
Recursing with N = 1
min_maxN called with N = 1
Returning best move: c1h6
Recursing with N = 1
min_maxN called with N = 1
Returning best move: c1h6
Recursing with N = 1
min_maxN called with N = 1
Returning best move: c1h6
Recursing with N = 1
min_maxN called with N = 1
Returning best move: c1g5
Recursing with N = 1
min_maxN called with N = 1
Returning best move: c1h6
Recursing with N = 1
min_maxN called with N = 1
Returning best move: c1h6
Recursing with N = 1
min_maxN called with N = 1
Returning best move: c1h6
Recursing with N = 1
min_maxN called with N = 1
Returni

min_maxN called with N = 2
Recursing with N = 1
min_maxN called with N = 1
Returning best move: d1d5
Recursing with N = 1
min_maxN called with N = 1
Returning best move: d1d5
Recursing with N = 1
min_maxN called with N = 1
Returning best move: d1d5
Recursing with N = 1
min_maxN called with N = 1
Returning best move: d1d5
Recursing with N = 1
min_maxN called with N = 1
Returning best move: d1d5
Recursing with N = 1
min_maxN called with N = 1
Returning best move: d1d5
Recursing with N = 1
min_maxN called with N = 1
Returning best move: d1d5
Recursing with N = 1
min_maxN called with N = 1
Returning best move: d1d5
Recursing with N = 1
min_maxN called with N = 1
Returning best move: f4g5
Recursing with N = 1
min_maxN called with N = 1
Returning best move: e1d2
Recursing with N = 1
min_maxN called with N = 1
Returning best move: d1d5
Recursing with N = 1
min_maxN called with N = 1
Returning best move: d1d5
Recursing with N = 1
min_maxN called with N = 1
Returning best move: d1d5
Recursing w

In [15]:
pygame.quit()

In [8]:
reader = chess.polyglot.open_reader('baron30.bin')


scoring= {'p': -1,
          'n': -3,
          'b': -3,
          'r': -5,
          'q': -9,
          'k': 0,
          'P': 1,
          'N': 3,
          'B': 3,
          'R': 5,
          'Q': 9,
          'K': 0,
          
          }

def eval_board(BOARD):
    score = 0
    pieces = BOARD.piece_map()
    for key in pieces:
        score += scoring[str(pieces[key])]

    return score

def eval_space(BOARD):
    no_moves = len(list(BOARD.legal_moves))

    #this function is always between 0 and 1 so we will never evaluate
    #this as being greater than a pawns value. The 20 value is arbitrary
    #but this number is chosen as it centers value around 0.5
    value = (no_moves/(20+no_moves))
    
    if BOARD.turn == True:
        return value
    else:
        return -value

def min_maxN(BOARD,N):

    opening_move = reader.get(BOARD)
    if opening_move == None:
        pass
    else:
        return opening_move.move

    #generate list of possible moves
    moves = list(BOARD.legal_moves)
    scores = []

    #score each move
    for move in moves:
        #temp allows us to leave the original game state unchanged
        temp = deepcopy(BOARD)
        temp.push(move)

        #here we must check that the game is not over
        outcome = temp.outcome()
        
        #if checkmate
        if outcome == None:
            #if we have not got to the final depth
            #we search more moves ahead
            if N>1:
                temp_best_move = min_maxN(temp,N-1)
                temp.push(temp_best_move)

            scores.append(eval_board(temp))

        #if checkmate
        elif temp.is_checkmate():

            # we return this as best move as it is checkmate
            return move

        # if stalemate
        else:
            #value to disencourage a draw
            #the higher the less likely to draw
            #default value should be 0
            #we often pick 0.1 to get the bot out of loops in bot vs bot
            val = 1000
            if BOARD.turn == True:
                scores.append(-val)
            else:
                scores.append(val)

        #this is the secondary eval function
        scores[-1] = scores[-1] + eval_space(temp)

    if BOARD.turn == True:
        best_move = moves[scores.index(max(scores))]
    else:
        best_move = moves[scores.index(min(scores))]

    return best_move

In [8]:
def min_max1(BOARD):
    return min_maxN(BOARD,1)

def min_max2(BOARD):
    return min_maxN(BOARD,2)

def min_max3(BOARD):
    return min_maxN(BOARD,3)

def min_max4(BOARD):
    return min_maxN(BOARD,4)

In [13]:
def min_max2(BOARD, reader):
    return min_maxN(BOARD, 2, reader)

In [7]:
scoring = {'p': -1, 'n': -3, 'b': -3, 'r': -5, 'q': -9, 'k': 0,
           'P': 1, 'N': 3, 'B': 3, 'R': 5, 'Q': 9, 'K': 0}

def eval_board(board):
    score = 0
    pieces = board.piece_map()
    for key in pieces:
        score += scoring[str(pieces[key])]

    return score

def eval_space(board):
    no_moves = len(list(board.legal_moves))
    value = (no_moves / (20 + no_moves))

    if board.turn:
        return value
    else:
        return -value

def min_maxN(board, N, reader):
    print("min_maxN called with N =", N)  # Debug message

    opening_move = reader.get(board)
    if opening_move is not None:
        print("Found opening move:", opening_move.move)  # Debug message
        return opening_move.move

    moves = list(board.legal_moves)
    scores = []

    for move in moves:
        temp = board.copy()
        temp.push(move)
        outcome = temp.outcome()

        if outcome is None:
            if N > 1:
                print("Recursing with N =", N-1)  # Debug message
                temp_best_move = min_maxN(temp, N-1, reader)
                temp.push(temp_best_move)

            scores.append(eval_board(temp))

        elif temp.is_checkmate():
            print("Checkmate found:", move)  # Debug message
            return move

        else:
            val = 1000
            if board.turn:
                scores.append(-val)
            else:
                scores.append(val)

        scores[-1] = scores[-1] + eval_space(temp)

    if board.turn:
        best_move = moves[scores.index(max(scores))]
    else:
        best_move = moves[scores.index(min(scores))]

    print("Returning best move:", best_move)  # Debug message
    return best_move

In [10]:


def main_two_agent(BOARD,agent1,agent_color1,agent2):
    '''
    for agent vs agent game
    
    '''
  
    #make background black
    scrn.fill(BLACK)
    #name window
    pygame.display.set_caption('Chess')
    
    #variable to be used later

    status = True
    while (status):
        #update screen
        update(scrn,BOARD)
        
        if BOARD.turn==agent_color1:
            BOARD.push(agent1(BOARD))

        else:
            BOARD.push(agent2(BOARD))

        scrn.fill(BLACK)
            
        for event in pygame.event.get():
     
            # if event object type is QUIT
            # then quitting the pygame
            # and program both.
            if event.type == pygame.QUIT:
                status = False
     
    # deactivates the pygame library
        if BOARD.outcome() != None:
            print(BOARD.outcome())
            status = False
            print(BOARD)
    pygame.quit()

In [11]:
def play_game(board, agent, agent_color, reader):  # Add 'reader' as an argument
    index_moves = []
    highlighted_squares = []

    # Main game loop
    while True:
        # Update the screen
        if not highlighted_squares:
            update(scrn, board)

        # Check for game over conditions
        if board.is_game_over():
            print("Game Over")
            print("Result: " + board.result())
            break

        # Agent's turn
        if board.turn == agent_color:
            move = agent(board, reader)  # Pass 'reader' to the agent function
            board.push(move)

        # Human player's turn
        else:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    return

                if event.type == pygame.MOUSEBUTTONDOWN:
                    update(scrn, board)
                    pos = pygame.mouse.get_pos()
                    square = (math.floor(pos[0] / 100), math.floor(pos[1] / 100))
                    index = (7 - square[1]) * 8 + square[0]
                    highlighted_squares = []

                    if index in index_moves:
                        move = moves[index_moves.index(index)]
                        board.push(move)
                        index_moves = []
                        highlighted_squares = []
                    else:
                        piece = board.piece_at(index)
                        if piece is not None:
                            all_moves = list(board.legal_moves)
                            moves = []
                            for m in all_moves:
                                if m.from_square == index:
                                    moves.append(m)
                                    t = m.to_square
                                    highlighted_squares.append((t % 8, 7 - t // 8))
                            index_moves = [m.to_square for m in moves]

        # Draw highlighted squares
        for square in highlighted_squares:
            TX1 = 100 * square[0]
            TY1 = 100 * square[1]
            pygame.draw.rect(scrn, BLUE, pygame.Rect(TX1, TY1, 100, 100), 5)

        pygame.display.flip()